In [2]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime

**A note on indexing**

In [3]:
#Let’s start with a DataFrame with two columns:
df = pd.DataFrame({"user_id": [1, 2, 3], "score": [10, 15, 20]})

In [4]:
df

,user_id,score
0,1,10
1,2,15
2,3,20
